# Swash Timestack Tracer
This notebook is used to train a U-Net to find the position of the swash front through a timestack. Once segmentation has been done a thresholding technique similar to Otsu's method is used to defined the shoreline.

# Your Task:
Train the best U-net for tracing the swash front. The default structure used is Wei Dai's example which you can build from by tuning hyperparameters.

Code cell 1 imports all the libraries and stores a function used for evaluating your trained model. You don't need to change anything in this code cell.

Code cell 2 loads the training images into the notebook and adds any data augmentation. 

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import shutil
import random
import csv
from scipy.stats import norm


# https://stackoverflow.com/questions/22579434/python-finding-the-intersection-point-of-two-gaussian-curves
def solve(m1,m2,std1,std2):
    a = 1/(2*std1**2) - 1/(2*std2**2)
    b = m2/(std2**2) - m1/(std1**2)
    c = m1**2 /(2*std1**2) - m2**2 / (2*std2**2) - np.log(std2/std1)
    return np.roots([a,b,c])

# function used to evaluate how good the model is
def compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir):
    
    folder_name = "test_timestacks/"

    image_range = max_uprush - min_uprush
    
    timestack = cv2.imread(folder_name + timestack_name)
    timestack_focus = timestack[:, min_uprush:max_uprush]
    timestack_labelled = cv2.imread(folder_name + timestack_name + "_output.png")
    timestack_annotated = timestack_labelled.copy()
    mask_annotated = cv2.imread(folder_name + timestack_name + "_output.png", cv2.IMREAD_GRAYSCALE)
    
    timestack_count = 0

    IMG_SIZE_WID = int(IMG_SIZE) # can change this if needed
    tf_images = []
    if tf_model_version == "michaelThompson_imgSize_64":
        # this way of reading/saving models worked for an older version of tensorflow 2.x
        tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")
    else:
        tf_model_xc = tf.keras.models.load_model(tf_model_version + "_model")

    vertical_coordinates = []
    horizontal_coordinates = []
    full_height, full_width, _ = timestack_focus.shape
    snap_width = full_width
    snap_height = full_width
    max_snap_start = full_height - snap_height
    snap_start = 0
    snap_counter = 0
    # split the image up into square sections - overlap the last section
    while snap_start < max_snap_start:

        # get square image
        snapshot = timestack_focus[snap_start:(snap_start + snap_height)]
        
        # prepare as tf input
        snapshot_resize = cv2.resize(snapshot, (IMG_SIZE_WID, IMG_SIZE))
        snapshot_resize_nm = np.array(snapshot_resize) / 255
        snapshot_input = snapshot_resize_nm.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)
        
        # put into model
        mask_test = tf_model_xc.predict([snapshot_input])[0]
        mask_test = cv2.resize(mask_test, (snap_width, snap_height))
        
        # add mask to mask image
        mask_annotated[snap_start:(snap_start + snap_height), min_uprush:max_uprush] = mask_test * 255
        
        # apply light and dark gaussian interesection method to extract shoreline points
        # use something similar to otsu's method for each row to draw a boundary line
        for j in range(0, len(mask_test), 1):
            pixel_row = mask_test[j]

            light_pixels = []
            dark_pixels = []
            for k in range(0, len(pixel_row), 1):
                if pixel_row[k] >= 0.5:
                    light_pixels.extend([k]*int(pixel_row[k]*255))
                else:
                    dark_pixels.extend([k]*int((0.5 - pixel_row[k])*255))

            # find mean and variance of pixels
            light_mean, light_std = norm.fit(light_pixels)
            dark_mean, dark_std = norm.fit(dark_pixels)

            try:
                # get the interect of both gaussians
                intersect_i = solve(light_mean, dark_mean, light_std, dark_std)

                # if intersect has more than 1 point, choose the point closest to the middle of the two means
                if len(intersect_i) == 2:
                    mean_middle = (light_mean + dark_mean)/2
                    intersect = min(intersect_i, key=lambda x:abs(x-mean_middle))
                # if 1 point thats great!
                elif len(intersect_i) == 1:
                    intersect = intersect_i
                # if 0 or 3 or more points, simply choose middle of two means
                else:
                    intersect = (light_mean + dark_mean)/2
            except:
                # just use middle of means
                if np.isnan(light_mean) or np.isnan(dark_mean):
                    intersect = 0
                else:
                    intersect = (light_mean + dark_mean)/2
            
            # annotate over timestack
            cv2.circle(timestack_annotated, (min_uprush + int(intersect), snap_start + j), 1, (255, 0, 0))
            
            # save vertical and horizontal coordinates for comparison with labels.
            horizontal_coordinates.append(snap_start + j)
            vertical_coordinates.append(min_uprush + int(intersect))
        
        timestack_count = timestack_count + 1
        snap_start = snap_start + int(snap_height)
        snap_counter = snap_counter + 1
    
    snap_end = snap_counter*snap_height
    # after while loop is finished, do one final snapshot that overlaps with last snapshot till the end.
    # get square image
    snapshot = timestack_focus[(full_height - snap_height):full_height]

    # prepare as tf input
    snapshot_resize = cv2.resize(snapshot, (IMG_SIZE_WID, IMG_SIZE))
    snapshot_resize_nm = np.array(snapshot_resize) / 255
    snapshot_input = snapshot_resize_nm.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)

    # put into model
    mask_test = tf_model_xc.predict([snapshot_input])[0]
    mask_test = cv2.resize(mask_test, (snap_width, snap_height))
    
    # add mask to mask image
    mask_annotated[(full_height - snap_height):full_height, min_uprush:max_uprush] = mask_test * 255
    
    # apply light and dark gaussian interesection method to extract shoreline points
    # use something similar to otsu's method for each row to draw a boundary line
    for j in range(0, len(mask_test), 1):
        pixel_row = mask_test[j]

        light_pixels = []
        dark_pixels = []
        for k in range(0, len(pixel_row), 1):
            if pixel_row[k] >= 0.5:
                light_pixels.extend([k]*int(pixel_row[k]*255))
            else:
                dark_pixels.extend([k]*int((0.5 - pixel_row[k])*255))

        # find mean and variance of pixels
        light_mean, light_std = norm.fit(light_pixels)
        dark_mean, dark_std = norm.fit(dark_pixels)

        try:
            # get the interect of both gaussians
            intersect_i = solve(light_mean, dark_mean, light_std, dark_std)
            
            # if intersect has more than 1 point, choose the point closest to the middle of the two means
            if len(intersect_i) == 2:
                mean_middle = (light_mean + dark_mean)/2
                intersect = min(intersect_i, key=lambda x:abs(x-mean_middle))
            # if 1 point thats great!
            elif len(intersect_i) == 1:
                intersect = intersect_i
            # if 0 or 3 or more points, simply choose middle of two means
            else:
                intersect = (light_mean + dark_mean)/2
        except:
            # just use middle of means
            if np.isnan(light_mean) or np.isnan(dark_mean):
                intersect = 0
            else:
                intersect = (light_mean + dark_mean)/2
        
        # only add to timestack and data if new data
        if (full_height - snap_height + j) >= snap_end:
            # annotate over timestack
            cv2.circle(timestack_annotated, (min_uprush + int(intersect), full_height - snap_height + j), 1, (255, 0, 0))

            # save vertical and horizontal coordinates for comparison with labels.
            horizontal_coordinates.append(full_height - snap_height + j)
            vertical_coordinates.append(min_uprush + int(intersect))
        
    # get csv values from labelled timestack
    manual_csv_points_filename = folder_name + "runup_data_test" + timestack_name + ".csv"
    x_vals = []
    t_vals = []
    with open(manual_csv_points_filename, newline='') as csvfile:
        points_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        initial_row = 5
        initial_row_counter = 0
        for row in points_reader:
            if initial_row_counter > initial_row:
                rows = row[0].split(',')
                x_vals.append( int(rows[0][1:-1]) ) # need to remove [] from value and turn into a number
                t_vals.append( int(rows[1][1:-1]) ) # need to remove [] from value and turn into a number
            initial_row_counter = initial_row_counter + 1

    print("red is hand picked, green is found by model")
    plt.plot(x_vals, t_vals, color = 'r')
    plt.plot(vertical_coordinates, horizontal_coordinates, color = 'g')
    plt.gca().invert_yaxis() # invert axis so it is like the timestack image coordinates
    plt.show()

    #full_stack_window = "timestack_annotated"
    #cv2.namedWindow(full_stack_window, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(full_stack_window, 1080, 1080)
    #cv2.imshow(full_stack_window, timestack_annotated)
    #cv2.waitKey()
    #cv2.destroyAllWindows()

    # find MSE for evey row
    SE = []
    for i in range(0, len(t_vals), 1):
        # make sure the comparison values line up correctly
        index_matching = np.where(np.array(horizontal_coordinates) == t_vals[i])
        vertical_coordinate_matching = vertical_coordinates[ index_matching[0][0] ]
        # /image range normalises the pixel values between 0 and 1
        SE.append( ((x_vals[i] - vertical_coordinate_matching)/image_range)**2 )
    MSE = np.mean(SE)

    # save the image
    print("saving image")
    cv2.imwrite(results_dir + "/" + "annotated_runup_timestack_" + timestack_name + "_" + tf_model_version + ".png", timestack_annotated)
    
    # save the image
    print("saving mask")
    cv2.imwrite(results_dir + "/" + "mask_output_timestack_" + timestack_name + "_" + tf_model_version + ".png", mask_annotated)
    
    # save the data
    print("Saving Data to csv file")

    # create csv file
    output_data_name = results_dir + "/" + "runup_data_" + timestack_name + "_" + tf_model_version + ".csv"
    with open(output_data_name, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows([["timestack shoreline coordinates"]])

    with open(output_data_name, 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)

        time_row = list(["time (horizontal coordinate)"])
        time_row.extend(horizontal_coordinates)
        x_row = list(["x (vertical coordinate)"])
        x_row.extend(vertical_coordinates)
        writer.writerow(time_row)
        writer.writerow(x_row)

    csvFile.close()

    return MSE

print("done")

Now upload images and masks

In [ ]:
"""%%%%%%%%%%%%%%%%%%%% GET IMAGES %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# Instead get my own images

input_images_folder = "colour_images"
mask_images_folder = "masks"

IMG_SIZE = 128 # smallest image size is 593x592
IMG_SIZE_WID = IMG_SIZE
brightness_change = 40 # only necessary if you include data augmentation with brightness variation

data = []
# first get all images and label accordingly
image_names = os.listdir(input_images_folder)
mask_names = os.listdir(mask_images_folder)
for i in range(0, len(image_names), 1):
    
    # data should be list of images and masks
    inp_img_full = cv2.imread(input_images_folder + "/" + image_names[i])
    
    # get equivalent mask name
    # this line matches the timestack name (0th to 22nd character) and snapshot number name (between m and .)
    mask_indx = mask_names.index(image_names[i][:23] + "_mask_" + image_names[i].split('m')[1].split('.')[0] + ".png")
    mask_label_gray_full = cv2.imread(mask_images_folder + "/" + mask_names[mask_indx], cv2.IMREAD_GRAYSCALE)
    
    # resize masks and colour images
    inp_img = cv2.resize(inp_img_full, (IMG_SIZE_WID, IMG_SIZE))
    mask_label_gray = cv2.resize(mask_label_gray_full, (IMG_SIZE_WID, IMG_SIZE))
    
    # convert mask into black and white
    mask_label = cv2.threshold(mask_label_gray, 127, 255, cv2.THRESH_BINARY)[1]
    
    # do some manipulations with brightness
    # https://stackoverflow.com/questions/37822375/python-opencv-increasing-image-brightness-without-overflowing-uint8-array
    # increase the brightness of image
    inp_img_l = inp_img.copy()
    inp_img_l = np.where((255 - inp_img_l) < brightness_change, 255, inp_img_l + brightness_change)

    # decrease the brightness of image
    inp_img_d = inp_img.copy()
    inp_img_d = np.where((inp_img_d) < brightness_change, 0, inp_img_d - brightness_change)

    data.append([inp_img_d, mask_label])  
    data.append([inp_img_l, mask_label])
    data.append([inp_img, mask_label])

    
random.seed(0) # good for comparing hyperparameter performance

# randomize rows of data
random.shuffle(data)

# setup a training set and test set
training_test_ratio = 0.90 # 90% training is because the more meaningful test set is another set of images. Otherwise this would be a larger proportion. You could even set this to 1 if you wanted.
data_train = data[0:int(training_test_ratio*len(data))]
data_test = data[int(training_test_ratio*len(data)):]

# split x data from y data
x_train = np.array(list(map(list, zip(*data_train)))[0])
y_train = np.array(list(map(list, zip(*data_train)))[1])

# needs to all be numpy arrays to work in tensorflow
x_test = np.array(list(map(list, zip(*data_test)))[0])
x_test_show = x_test # keep a copy for visualization later
y_test = np.array(list(map(list, zip(*data_test)))[1])

# normalize input features (which are the pixel values)
#x_train = tf.keras.utils.normalize(x_train, axis=1)
#x_test = tf.keras.utils.normalize(x_test, axis=1)
#y_train = tf.keras.utils.normalize(y_train, axis=1)
#y_test = tf.keras.utils.normalize(y_test, axis=1)
x_train = x_train / 255
x_test = x_test / 255
y_train = y_train / 255
y_test = y_test / 255

# reshape for convolutional compatability
x_train = x_train.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3) # 3 is for the 3 colour channels
x_test = x_test.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)
y_train = y_train.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 1) # this is for the black and white
y_test = y_test.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 1)

print("done")

Unet Architecture - This is from Wei Dai's example U-net python code from the Summer of AI 2020. Slightly modified for this use case.

In [ ]:
"""%%%%%%%%%%%%%%%%%%%% MACHINE LEARNING %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
# Every time you train a model, change the name of tf_model_version below. 
# It is also good practice to record all hyperparameters you used and the performance 
# of each model trained. Then you can look through all your results and learn something.
tf_model_version = "test_0"

def unet_model(output_channels = 1, f=8):
    inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE_WID, 3))
    
    # Downsampling through the model
    d1 = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(inputs)
    d1 = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(d1)

    d2 = tf.keras.layers.MaxPooling2D()(d1)
    d2 = tf.keras.layers.Conv2D(2*f, 3, padding='same', activation='relu')(d2)
    d2 = tf.keras.layers.Conv2D(2*f, 3, padding='same', activation='relu')(d2)
    
    d3 = tf.keras.layers.MaxPooling2D()(d2)
    d3 = tf.keras.layers.Conv2D(4*f, 3, padding='same', activation='relu')(d3)
    d3 = tf.keras.layers.Conv2D(4*f, 3, padding='same', activation='relu')(d3)

    d4 = tf.keras.layers.MaxPooling2D()(d3)
    d4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(d4)
    d4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(d4)

    d5 = tf.keras.layers.MaxPooling2D()(d4)
    d5 = tf.keras.layers.Conv2D(16*f, 3, padding='same', activation='relu')(d5)
    d5 = tf.keras.layers.Conv2D(16*f, 3, padding='same', activation='relu')(d5)

    # Upsampling and establishing the skip connections
    u4 = tf.keras.layers.UpSampling2D()(d5)
    u4 = tf.keras.layers.concatenate([u4, d4])
    u4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(u4)
    u4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(u4)

    u3 = tf.keras.layers.UpSampling2D()(u4)
    u3 = tf.keras.layers.concatenate([u3, d3])
    u3 = tf.keras.layers.Conv2D(4*f, 3, padding='same', activation='relu')(u3)
    u3 = tf.keras.layers.Conv2D(4*f, 3, padding='same', activation='relu')(u3)

    u2 = tf.keras.layers.UpSampling2D()(u3)
    u2 = tf.keras.layers.concatenate([u2, d2])
    u2 = tf.keras.layers.Conv2D(2*f, 3, padding='same', activation='relu')(u2)
    u2 = tf.keras.layers.Conv2D(2*f, 3, padding='same', activation='relu')(u2)

    u1 = tf.keras.layers.UpSampling2D()(u2)
    u1 = tf.keras.layers.concatenate([u1, d1])
    u1 = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(u1)
    u1 = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(u1)

    # This is the last layer of the model. 
    outputs = tf.keras.layers.Conv2D(output_channels, 1, activation='sigmoid')(u1)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

model = unet_model(1,8)
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=1, validation_split=0.1, epochs=10)

"""%%%%%%%%%%%%%%%%%%%% EVALUATION %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""

# validation loss and validation accuracy should be close to training loss and accuracy (otherwise overfitting!)
train_loss, train_acc = model.evaluate(x_train, y_train)
val_loss, val_acc = model.evaluate(x_test, y_test)

print("training loss = " + str(train_loss) + ", training accuracy = " + str(train_acc))
print("test loss     = " + str(val_loss) + ", test accuracy     = " + str(val_acc))

#model.save(tf_model_version + ".model") # this worked for an older version of tensorflow 2.x
model.save(tf_model_version + "_model")

predictions = model.predict([x_test])

#print(predictions[0])

# plot graphs inside the notebook
%matplotlib inline 

# make a new directory for results
current_path = os.getcwd()
results_dir = tf_model_version
results_dir_path = os.path.join(current_path, results_dir)
if os.path.exists(results_dir_path):
    shutil.rmtree(results_dir_path)
    print("old results directory removed")
os.mkdir(results_dir_path)
print("new results directory created called " + results_dir)

# don't change the min_uprush and max_uprush values, 
# these are fixed values according to the labelled evaluation timestacks
error_list = []
timestack_name = "20101109084810_02_25ppm_test_s.png" # 194 and 925
min_uprush = 194
max_uprush = 925
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "20101110094203_05_25ppm_test_s.png" # 40 and 580
min_uprush = 40
max_uprush = 580
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "20101111092821_01_25ppm_test_s.png" # 116 and 800
min_uprush = 116
max_uprush = 800
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "OneMile8_test_s.png" # 77 and 289
min_uprush = 77
max_uprush = 289
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "FC4_test_s.png" # 37 and 240
min_uprush = 37
max_uprush = 240
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

print("\nMSE summary for all timestacks:")
print("20101109084810_02_25ppm_test_s.png" + " " + str(error_list[0]))
print("20101110094203_05_25ppm_test_s.png" + " " + str(error_list[1]))
print("20101111092821_01_25ppm_test_s.png" + " " + str(error_list[2]))
print("OneMile8_test_s.png               " + " " + str(error_list[3]))
print("FC4_test_s.png                    " + " " + str(error_list[4]))

print("\nAverage MSE for all seen timestacks = " + str(np.mean(error_list)))

print("done")

You can now visualize your results in the results directory. You can run the below cell to see what my best results were using a regression CNN.

In [ ]:
# function used to evaluate how good the model is
def compare_shoreline_mt(min_uprush, max_uprush, tf_model_version, IMG_SIZE, timestack_name, results_dir):
    
    folder_name = "test_timestacks/"

    image_range = max_uprush - min_uprush
    
    timestack = cv2.imread(folder_name + timestack_name)
    timestack_focus = timestack[:, min_uprush:max_uprush]
    timestack_labelled = cv2.imread(folder_name + timestack_name + "_output.png")
    timestack_annotated = timestack_labelled.copy()

    height_of_two_peaks = 104 # normally 104 for 2 peaks
    # for every row, find the shoreline location (plot as blue dots). If it is a minimum point plot as red Dot, If maximum plot as blue Dot
    padding = height_of_two_peaks
    height, width, _ = timestack.shape
    width = max_uprush - min_uprush
    timestack_count = 0

    IMG_SIZE_WID = int(IMG_SIZE) # can change this if needed
    tf_images = []
    if tf_model_version == "michaelThompson_imgSize_64":
        # this way of reading/saving models worked for an older version of tensorflow 2.x
        tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")
    else:
        tf_model_xc = tf.keras.models.load_model(tf_model_version + "_model")

    vertical_coordinates = []
    horizontal_coordinates = []
    for i in range(0, int((height - padding)), 1):

        horizontal_coordinate = i + int(padding/2)

        # get the small section of the image for the horizontal position i
        timestack_snapshot = timestack_focus[i:i + padding, :]
        title_window_2 = "Snapshot of timestack used for processing"

        # classify the timestack snapshot through the convolutional neural network for X_coordinate
        #tf_images = cv2.cvtColor(timestack_snapshot, cv2.COLOR_BGR2GRAY)   # should have this for gray
        tf_images = cv2.resize(timestack_snapshot, (IMG_SIZE_WID, IMG_SIZE))
        tf_images_i = np.array(tf_images)
        tf_images_i = tf.keras.utils.normalize(tf_images_i, axis=1)
        tf_images_i = tf_images_i.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)   #should be changed to 1 for gray
        prediction = tf_model_xc.predict([tf_images_i])

        vertical_coordinate = int((prediction[0])*width) + min_uprush

        cv2.circle(timestack_annotated, (vertical_coordinate, horizontal_coordinate), radius=1, color=[255, 0, 0])
        vertical_coordinates.append(vertical_coordinate)
        horizontal_coordinates.append(horizontal_coordinate)

        timestack_count = timestack_count + 1


    # get csv values from labelled timestack
    manual_csv_points_filename = folder_name + "runup_data_test" + timestack_name + ".csv"
    x_vals = []
    t_vals = []
    with open(manual_csv_points_filename, newline='') as csvfile:
        points_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        initial_row = 5
        initial_row_counter = 0
        for row in points_reader:
            if initial_row_counter > initial_row:
                rows = row[0].split(',')
                x_vals.append( int(rows[0][1:-1]) ) # need to remove [] from value and turn into a number
                t_vals.append( int(rows[1][1:-1]) ) # need to remove [] from value and turn into a number
            initial_row_counter = initial_row_counter + 1

    print("red is hand picked, green is found by model")
    plt.plot(x_vals, t_vals, color = 'r')
    plt.plot(vertical_coordinates, horizontal_coordinates, color = 'g')
    plt.gca().invert_yaxis() # invert axis so it is like the timestack image coordinates
    plt.show()

    #full_stack_window = "timestack_annotated"
    #cv2.namedWindow(full_stack_window, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(full_stack_window, 1080, 1080)
    #cv2.imshow(full_stack_window, timestack_annotated)
    #cv2.waitKey()
    #cv2.destroyAllWindows()

    # find MSE for evey row
    SE = []
    for i in range(0, len(t_vals), 1):
        # make sure the comparison values line up correctly
        index_matching = np.where(np.array(horizontal_coordinates) == t_vals[i])
        vertical_coordinate_matching = vertical_coordinates[ index_matching[0][0] ]
        # /image range normalises the pixel values between 0 and 1
        SE.append( ((x_vals[i] - vertical_coordinate_matching)/image_range)**2 )
    MSE = np.mean(SE)

    # save the image
    print("saving image")
    cv2.imwrite(results_dir + "/" + "annotated_runup_timestack_" + timestack_name + "_" + tf_model_version + ".png", timestack_annotated)
    
    # save the data
    print("Saving Data to csv file")

    # create csv file
    output_data_name = results_dir + "/" + "runup_data_" + timestack_name + "_" + tf_model_version + ".csv"
    with open(output_data_name, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows([["timestack shoreline coordinates"]])

    with open(output_data_name, 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)

        time_row = list(["time (horizontal coordinate)"])
        time_row.extend(horizontal_coordinates)
        x_row = list(["x (vertical coordinate)"])
        x_row.extend(vertical_coordinates)
        writer.writerow(time_row)
        writer.writerow(x_row)

    csvFile.close()

    return MSE

print("done")

mt_model = "michaelThompson_imgSize_64"
IMG_SIZE = 64

# make a new directory
current_path = os.getcwd()
results_dir = mt_model
results_dir_path = os.path.join(current_path, results_dir)
if os.path.exists(results_dir_path):
    shutil.rmtree(results_dir_path)
    print("old results directory removed")
os.mkdir(results_dir_path)
print("new results directory created called " + results_dir)

# don't change the min_uprush and max_uprush values, 
# these are fixed values according to the labelled evaluation timestacks
error_list = []
timestack_name = "20101109084810_02_25ppm_test_s.png" # 194 and 925
min_uprush = 194
max_uprush = 925
ground_truth_error = compare_shoreline_mt(min_uprush, max_uprush, mt_model, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "20101110094203_05_25ppm_test_s.png" # 40 and 580
min_uprush = 40
max_uprush = 580
ground_truth_error = compare_shoreline_mt(min_uprush, max_uprush, mt_model, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "20101111092821_01_25ppm_test_s.png" # 116 and 800
min_uprush = 116
max_uprush = 800
ground_truth_error = compare_shoreline_mt(min_uprush, max_uprush, mt_model, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "OneMile8_test_s.png" # 77 and 289
min_uprush = 77
max_uprush = 289
ground_truth_error = compare_shoreline_mt(min_uprush, max_uprush, mt_model, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "FC4_test_s.png" # 37 and 240
min_uprush = 37
max_uprush = 240
ground_truth_error = compare_shoreline_mt(min_uprush, max_uprush, mt_model, IMG_SIZE, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

print("\nMSE summary for all timestacks:")
print("20101109084810_02_25ppm_test_s.png" + " " + str(error_list[0]))
print("20101110094203_05_25ppm_test_s.png" + " " + str(error_list[1]))
print("20101111092821_01_25ppm_test_s.png" + " " + str(error_list[2]))
print("OneMile8_test_s.png               " + " " + str(error_list[3]))
print("FC4_test_s.png                    " + " " + str(error_list[4]))

print("\nAverage MSE for all seen timestacks = " + str(np.mean(error_list)))

print("done")

If you want to use a GPU, this code is helpful to check that it will work. It can be a bit of a pain to set it up in anaconda, but the efficiency is worth it!

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)